So far we have been happy with the way Pydantic serializes field values.

But sometimes, especially with certain data types, like datetimes, we may want to control how fields get serialized.

# Custom Serializers

We'll need to use a **decorator** function provided by Pydantic, called `@field_serializer` which is used to control serialization at the field level.

In [31]:
from pydantic import BaseModel, field_serializer

The decorator has several arguments that defines which field the serializer applies to and how the serializer needs to be applied.

One important option is:
- `when_used`: by default the custom serializer is always used, but we have other options available:
    - `always`: the default, serializer is executed when serializing either to a dict or to JSON
    - `unless-none`: serializer is not used if the value is None
    - `json`: serializer is only used when serializing to JSON
    - `json-unless-none`: serializer used when serializing to JSON, unless the value is None

There is also another option for mode plain vs wrap, but this is rarely used, and I won't cover it in this course.

Let's take a look the `when_used` option and understand the circumstances when our serializer gets called.

In [32]:
from datetime import datetime


class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="always")
    def serialize_name(self, value):
        print(f"type = {type(value)}")
        return value


m = Model(dt="2020-01-01T12:00:00")

print(m.model_dump())
print(m.model_dump_json())

type = <class 'datetime.datetime'>
{'dt': datetime.datetime(2020, 1, 1, 12, 0)}
type = <class 'datetime.datetime'>
{"dt":"2020-01-01T12:00:00"}


The data was correctly serialized, since Pydantic can correctly serialize datetime to JSON (uses ISO standard).

However, when serializing to JSON we may not want this datetime representation. 

We'll get back to that in a minute.

Let's see what happens if the value of `dt` is None:

In [33]:
m = Model()
print(m.model_dump())
print(m.model_dump_json())

type = <class 'NoneType'>
{'dt': None}
type = <class 'NoneType'>
{"dt":null}


As you can see, our custom serializer was called in both cases.

If we don't want to run our custom serializer when the field value is `None`, we can use one of the other `when_used` options:

In [34]:

from datetime import datetime


class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="unless-none")
    def serialize_name(self, value):
        print(f"type = {type(value)}")
        return value


m = Model(dt="2020-01-01T12:00:00")
print(m.model_dump())
null_model = Model()
print(null_model.model_dump())

type = <class 'datetime.datetime'>
{'dt': datetime.datetime(2020, 1, 1, 12, 0)}
{'dt': None}


As you can see, our serializer did not get called when `dt` was `None`.

Let's go back to the case where we only want to change the serialization when serializing to JSON. We might be OK with the dictionary serialization, but for our JSON output we want to modify the datetime format to be formatted like this:

```
2020/1/1 12:00 PM
```

So, let's use this in our serializer, and configure the serializer to only apply to JSON serialization, and not when the value is None:

In [35]:
from datetime import datetime


class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_name(self, value):
        print(f"type = {type(value)}")
        return value.strftime("%Y/%-m/%-d %I:%M %p")

In [36]:
m = Model(dt="2020-01-01T12:00:00")
m_null = Model()

print(m)
print(m.model_dump())
print(m.model_dump_json())
print(m_null.model_dump_json())

dt=datetime.datetime(2020, 1, 1, 12, 0)
{'dt': datetime.datetime(2020, 1, 1, 12, 0)}
type = <class 'datetime.datetime'>
{"dt":"2020/1/1 12:00 PM"}
{"dt":null}


In [37]:
m.model_dump_json()

type = <class 'datetime.datetime'>


'{"dt":"2020/1/1 12:00 PM"}'

---
Now suppose we want to implement a different serialization depending on whether we are serializing to a dictionary or to JSON.

We need to somehow be able to figure out, inside our serializer which serialization we are performing and react accordingly.

Pydantic implements yet another argument that we can add to our serializer function - an argument with type `FieldSerializationInfo`. Let's take a look:

In [48]:
from pydantic import FieldSerializationInfo


class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="unless-none")
    def dt_serializer(self, value, info: FieldSerializationInfo):
        print(f"info={info}")
        return value

In [50]:
m = Model(dt=datetime(2020, 1, 1))
m.model_dump()

info=SerializationInfo(include=None, exclude=None, mode='python', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False)


{'dt': datetime.datetime(2020, 1, 1, 0, 0)}

Notice that `mode` value in the `info` object? It is set to `python`.

Now, let's dump to JSON:

In [51]:
m.model_dump_json()

info=SerializationInfo(include=None, exclude=None, mode='json', by_alias=False, exclude_unset=False, exclude_defaults=False, exclude_none=False, round_trip=False)


'{"dt":"2020-01-01T00:00:00"}'

Notice that the `mode` is now set to `json`.

We could use that, but `FieldSerializationInfo` offers us a method named `mode_is_json` that we can use instead.

In [55]:
class Model(BaseModel):
    dt: datetime | None = None

    @field_serializer("dt", when_used="unless-none")
    def dt_serializer(self, value, info: FieldSerializationInfo):
        print(f"mode_is_json={info.mode_is_json()}")
        return value

In [56]:
m = Model(dt=datetime(2020, 1, 1))

In [58]:
m.model_dump()

mode_is_json=False


{'dt': datetime.datetime(2020, 1, 1, 0, 0)}

In [59]:
m.model_dump_json()

mode_is_json=True


'{"dt":"2020-01-01T00:00:00"}'

Let's write a simple Python function that will do the following, given a datetime object as an argument:
- if the datetime is naive, make it aware, and assume the naive datetime was already UTC
- if the datetime is aware, change it to be UTC